### Akin - Example Usage

In [1]:
from akin import MultiHash, LSH
import pandas as pd

In [2]:
content = pd.DataFrame(
    {
        'text': [
            'Jupiter is primarily composed of hydrogen with a quarter of its mass being helium',
            'Jupiter moving out of the inner Solar System would have allowed the formation of inner planets.',
            'A helium atom has about four times as much mass as a hydrogen atom, so the composition changes '
            'when described as the proportion of mass contributed by different atoms.',
            'Jupiter is primarily composed of hydrogen and a quarter of its mass being helium',
            'A helium atom has about four times as much mass as a hydrogen atom and the composition changes '
            'when described as a proportion of mass contributed by different atoms.',
            'Theoretical models indicate that if Jupiter had much more mass than it does at present, it '
            'would shrink.',
            'This process causes Jupiter to shrink by about 2 cm each year.',
            'Jupiter is mostly composed of hydrogen with a quarter of its mass being helium',
            'The Great Red Spot is large enough to accommodate Earth within its boundaries.'
        ]
    }
)

**Create MinHash object:**

In [3]:
minhash = MultiHash(n_gram=9, permutations=100, hash_bits=64, seed=3)

In [4]:
signatures = minhash.transform(content['text'])

In [5]:
content['signature'] = signatures

In [6]:
content

,text,signature
0,Jupiter is primarily composed of hydrogen with...,"(-9050934246571064385, -9214906801195985462, -..."
1,Jupiter moving out of the inner Solar System w...,"(-9174302289558572688, -9194971056948479927, -..."
2,A helium atom has about four times as much mas...,"(-9175048834891487841, -9085998147126773419, -..."
3,Jupiter is primarily composed of hydrogen and ...,"(-8798265333996157434, -8962219986627421974, -..."
4,A helium atom has about four times as much mas...,"(-9175048834891487841, -9127105545562189603, -..."
5,Theoretical models indicate that if Jupiter ha...,"(-9006657039299929325, -9174745613751414767, -..."
6,This process causes Jupiter to shrink by about...,"(-9093545881359078396, -9149786975706429297, -..."
7,Jupiter is mostly composed of hydrogen with a ...,"(-9050934246571064385, -9214906801195985462, -..."
8,The Great Red Spot is large enough to accommod...,"(-9220181301477941095, -9070612480513123769, -..."


**Create LSH object:**

In [7]:
lsh = LSH(no_of_bands=50)

In [8]:
lsh.update(signatures)

**Query to find near duplicates for text 1:**

In [9]:
text_1_minhash = signatures[0]

In [10]:
near_duplicates = lsh.query(text_1_minhash, min_jaccard=0.5)

In [11]:
content.loc[content['signature'].isin(near_duplicates)]['text']

3    Jupiter is primarily composed of hydrogen and ...
7    Jupiter is mostly composed of hydrogen with a ...
Name: text, dtype: object

**Generate minhash signature and add new texts to LSH model:**

In [12]:
new_text = [
    'Jupiter is primarily composed of hydrogen with a quarter of its mass being helium',
    'Jupiter moving out of the inner Solar System would have allowed the formation of '
    'inner planets.'
]

In [13]:
new_signatures = minhash.transform(new_text)
lsh.update(new_signatures)

**Remove text and label from model:**

In [14]:
# Remove text and label from model.
lsh.remove([signatures[3]])

In [15]:
near_duplicates = lsh.query(text_1_minhash, min_jaccard=0.5)
content.loc[content['signature'].isin(near_duplicates)]['text']

7    Jupiter is mostly composed of hydrogen with a ...
Name: text, dtype: object